In [72]:
from crewai import Agent, Task, Crew
from langchain_ollama import ChatOllama
import os
from langchain.agents import tool
from textwrap import dedent
from exa_py import Exa



In [73]:
EXA_API_KEY="ae1c4cc7-a242-4e3e-909e-da0d6c87f41a"

In [74]:
class MeetingPreparationAgents():
	def research_agent(self):
		return Agent(
			role='Research Specialist',
			goal='Conduct thorough research on people and companies involved in the meeting',
			tools=ExaSearchTool.tools(),
			backstory=dedent("""\
					As a Research Specialist, your mission is to uncover detailed information
					about the individuals and entities participating in the meeting. Your insights
					will lay the groundwork for strategic meeting preparation."""),
			verbose=True,llm = ChatOllama(model = "llama3",base_url = "http://localhost:11434")
		)

	def industry_analysis_agent(self):
		return Agent(
			role='Industry Analyst',
			goal='Analyze the current industry trends, challenges, and opportunities',
			tools=ExaSearchTool.tools(),
			backstory=dedent("""\
					As an Industry Analyst, your analysis will identify key trends,
					challenges facing the industry, and potential opportunities that
					could be leveraged during the meeting for strategic advantage."""),
			verbose=True,llm = ChatOllama(model = "llama3",base_url = "http://localhost:11434")
		)

	def meeting_strategy_agent(self):
		return Agent(
			role='Meeting Strategy Advisor',
			goal='Develop talking points, questions, and strategic angles for the meeting',
			tools=ExaSearchTool.tools(),
			backstory=dedent("""\
					As a Strategy Advisor, your expertise will guide the development of
					talking points, insightful questions, and strategic angles
					to ensure the meeting's objectives are achieved."""),
			verbose=True,llm = ChatOllama(model = "llama3",base_url = "http://localhost:11434")
		)

	def summary_and_briefing_agent(self):
		return Agent(
			role='Briefing Coordinator',
			goal='Compile all gathered information into a concise, informative briefing document',
			tools=ExaSearchTool.tools(),
			backstory=dedent("""\
					As the Briefing Coordinator, your role is to consolidate the research,
					analysis, and strategic insights."""),
			verbose=True,llm = ChatOllama(model = "llama3",base_url = "http://localhost:11434")
		)

In [75]:
class MeetingPreparationTasks():
	def research_task(self, agent, participants, context):
		return Task(
			description=dedent(f"""\
				Conduct comprehensive research on each of the individuals and companies
				involved in the upcoming meeting. Gather information on recent
				news, achievements, professional background, and any relevant
				business activities.

				Participants: {participants}
				Meeting Context: {context}"""),
			expected_output=dedent("""\
				A detailed report summarizing key findings about each participant
				and company, highlighting information that could be relevant for the meeting."""),
			async_execution=True,
			agent=agent
		)

	def industry_analysis_task(self, agent, participants, context):
		return Task(
			description=dedent(f"""\
				Analyze the current industry trends, challenges, and opportunities
				relevant to the meeting's context. Consider market reports, recent
				developments, and expert opinions to provide a comprehensive
				overview of the industry landscape.

				Participants: {participants}
				Meeting Context: {context}"""),
			expected_output=dedent("""\
				An insightful analysis that identifies major trends, potential
				challenges, and strategic opportunities."""),
			async_execution=True,
			agent=agent
		)

	def meeting_strategy_task(self, agent, context, objective):
		return Task(
			description=dedent(f"""\
				Develop strategic talking points, questions, and discussion angles
				for the meeting based on the research and industry analysis conducted

				Meeting Context: {context}
				Meeting Objective: {objective}"""),
			expected_output=dedent("""\
				Complete report with a list of key talking points, strategic questions
				to ask to help achieve the meetings objective during the meeting."""),
			agent=agent
		)

	def summary_and_briefing_task(self, agent, context, objective):
		return Task(
			description=dedent(f"""\
				Compile all the research findings, industry analysis, and strategic
				talking points into a concise, comprehensive briefing document for
				the meeting.
				Ensure the briefing is easy to digest and equips the meeting
				participants with all necessary information and strategies.

				Meeting Context: {context}
				Meeting Objective: {objective}"""),
			expected_output=dedent("""\
				A well-structured briefing document that includes sections for
				participant bios, industry overview, talking points, and
				strategic recommendations."""),
			agent=agent
		)

In [76]:
tasks = MeetingPreparationTasks()
agents = MeetingPreparationAgents()

In [77]:
print("## Welcome to the Meeting Prep Crew")
print('-------------------------------')
participants = input("What are the emails for the participants (other than you) in the meeting?\n")
context = input("What is the context of the meeting?\n")
objective = input("What is your objective for this meeting?\n")


## Welcome to the Meeting Prep Crew
-------------------------------


What are the emails for the participants (other than you) in the meeting?
 sweta.rani19@gmail.com
What is the context of the meeting?
 API Management Product Roadmap Session
What is your objective for this meeting?
 Come up with AHA ideas that can be part of API Management Product Roadmap


In [78]:
class ExaSearchTool:
	@tool
	def search(query: str):
		"""Search for a webpage based on the query."""
		return ExaSearchTool._exa().search(f"{query}", use_autoprompt=True, num_results=3)

	@tool
	def find_similar(url: str):
		"""Search for webpages similar to a given URL.
		The url passed in should be a URL returned from `search`.
		"""
		return ExaSearchTool._exa().find_similar(url, num_results=3)

	@tool
	def get_contents(ids: str):
		"""Get the contents of a webpage.
		The ids must be passed in as a list, a list of ids returned from `search`.
		"""
		ids = eval(ids)
		contents = str(ExaSearchTool._exa().get_contents(ids))
		print(contents)
		contents = contents.split("URL:")
		contents = [content[:1000] for content in contents]
		return "\n\n".join(contents)

	def tools():
		return [ExaSearchTool.search, ExaSearchTool.find_similar, ExaSearchTool.get_contents]

	def _exa():
		return Exa(api_key=EXA_API_KEY)

In [79]:
# Create Agents
researcher_agent = agents.research_agent()
industry_analyst_agent = agents.industry_analysis_agent()
meeting_strategy_agent = agents.meeting_strategy_agent()
summary_and_briefing_agent = agents.summary_and_briefing_agent()

# Create Tasks
research = tasks.research_task(researcher_agent, participants, context)
industry_analysis = tasks.industry_analysis_task(industry_analyst_agent, participants, context)
meeting_strategy = tasks.meeting_strategy_task(meeting_strategy_agent, context, objective)
summary_and_briefing = tasks.summary_and_briefing_task(summary_and_briefing_agent, context, objective)

meeting_strategy.context = [research, industry_analysis]
summary_and_briefing.context = [research, industry_analysis, meeting_strategy]


In [80]:
# Create Tasks
research = tasks.research_task(researcher_agent, participants, context)
industry_analysis = tasks.industry_analysis_task(industry_analyst_agent, participants, context)
meeting_strategy = tasks.meeting_strategy_task(meeting_strategy_agent, context, objective)
summary_and_briefing = tasks.summary_and_briefing_task(summary_and_briefing_agent, context, objective)

meeting_strategy.context = [research, industry_analysis]
summary_and_briefing.context = [research, industry_analysis, meeting_strategy]


In [81]:
# Create Crew responsible for Copy
crew = Crew(
	agents=[
		researcher_agent,
		industry_analyst_agent,
		meeting_strategy_agent,
		summary_and_briefing_agent
	],
	tasks=[
		research,
		industry_analysis,
		meeting_strategy,
		summary_and_briefing
	]
)


2024-08-19 02:09:14,131 - 8514124480 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
game = crew.kickoff()




> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
Thought: I need to start by searching for information about Sweta Rani, the participant involved in the meeting.

Action: search
Action Input: {"query": "Sweta Rani 

Title: Ragani
URL: https://raganiworld.com/
ID: https://raganiworld.com/
Score: 0.15423956513404846
Published Date: 2020-01-01T00:00:00.000Z
Author: None
Text: None
Highlights: None
Highlight Scores: None
Summary: None


Title: K. Rani (singer) - Wikipedia
URL: https://en.wikipedia.org/wiki/K._Rani_(singer)
ID: https://en.wikipedia.org/wiki/K._Rani_(singer)
Score: 0.15160243213176727
Published Date: 2015-09-24T00:00:00.000Z
Author: Authority control MusicBrainz artist
Text: None
Highlights: None
Highlight Scores: None
Summary: None


Title: Baby Rani - Wikipedia
URL: https://en.wikipedia.org/wiki/Baby_Rani_(child_artist)
ID: https://en.wikipedia.org/wiki/Baby_Rani_(child_artist)
Score: 0.15016308426856995
Published Date: 2019-06-16T00: